In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
from matplotlib_venn import venn2
import seaborn as sns

sns.set_style()
plt.rcParams["figure.figsize"] = (12, 6)

In [2]:
# Lee egresos hospitalarios y consultas ambulatorias
df_egresos = pd.read_csv("../data/processed/df_procesada.csv")
df_consultas = pd.read_csv(
    "../data/external/datos_limpios_diagnosticos.csv", encoding="latin-1", sep=";"
)

C:\Users\ppizarro\AppData\Local\Temp\ipykernel_7580\1883192389.py:2: DtypeWarning: Columns (18,80,93,94,95,97,99,100,101,144,152,153,156,157,158) have mixed types. Specify dtype option on import or set low_memory=False.
  df_egresos = pd.read_csv("../data/processed/df_procesada.csv")


## Obtención de pacientes ambulatorios que tuvieron una hospitalización

In [4]:
# Obtiene los pacientes de ambulatorio, que hayan aparecido alguna vez en hospitalizados
pacientes_amb_con_hosp = df_consultas.merge(
    df_egresos, how="left", left_on="id_paciente", right_on="RUT"
)

# Calcula la cantidad de pacientes ambulatorios totales, y los que aparecieron en hospitalizados
conteo_de_pacientes_ambulatorios_con_hosp = pacientes_amb_con_hosp.groupby(
    "nombre_especialidad"
).agg(
    n_pacientes_ambulatorios=("id_paciente", "nunique"),
    n_pacientes_que_estuvieron_en_hosp=("RUT", "nunique"),
)

## Obtención de pacientes hospitalizados que tuvieron una consulta

In [5]:
# Une los pacientes hospitalizados con los ambulatorios
pacientes_hosp_con_amb = df_egresos.merge(
    df_consultas, how="left", left_on="RUT", right_on="id_paciente"
)

# Realiza el conteo de pacientes por especialidad
conteo_de_pacientes_hosp_con_amb = pacientes_hosp_con_amb.groupby("ESPEC").agg(
    n_pacientes_hospitalizados=("RUT", "nunique"),
    n_pacientes_hosp_con_consulta=("id_paciente", "nunique"),
)

## Obtencion de pacientes hospitalizados y ambulatorios por diagnostico

In [6]:
# Merge the outpatient and inpatient dataframes by diagnostic code and patient ID
pacientes_amb_y_hosp_por_diagnostico = df_consultas.merge(
    df_egresos,
    how="outer",
    left_on=["codigo_diagnostico", "id_paciente"],
    right_on=["DIAG1", "RUT"],
)

# Consolidate diagnostic codes, prioritizing inpatient diagnostics
pacientes_amb_y_hosp_por_diagnostico["diagnostico_consolidado"] = (
    pacientes_amb_y_hosp_por_diagnostico["DIAG1"].combine_first(
        pacientes_amb_y_hosp_por_diagnostico["codigo_diagnostico"]
    )
)

# Count unique outpatients and inpatients per diagnostic code
pacientes_amb_y_hosp_unicos_por_diagnostico = pacientes_amb_y_hosp_por_diagnostico.groupby(
    "diagnostico_consolidado"
).agg(
    pacientes_ambulatorios=("id_paciente", "nunique"), pacientes_hospitalizados=("RUT", "nunique")
)

# Identify patients with both a consultation and hospitalization
pacientes_con_consulta_y_hosp_por_diagnostico = pacientes_amb_y_hosp_por_diagnostico[
    pacientes_amb_y_hosp_por_diagnostico["id_paciente"].notna()
    & pacientes_amb_y_hosp_por_diagnostico["RUT"].notna()
]

# Count unique patients with both a consultation and hospitalization per diagnostic code
pacientes_con_consulta_y_hosp_por_diagnostico = (
    pacientes_con_consulta_y_hosp_por_diagnostico.groupby("diagnostico_consolidado").agg(
        pacientes_con_consulta_y_hosp=("id_paciente", "nunique")
    )
)

# Merge the unique outpatient/inpatient counts with the counts of patients with both
resumen_pacientes_hosp_y_amb = pacientes_amb_y_hosp_unicos_por_diagnostico.merge(
    pacientes_con_consulta_y_hosp_por_diagnostico, how="left", left_index=True, right_index=True
).fillna(0)

# Calculate the total number of unique patients per diagnostic code
resumen_pacientes_hosp_y_amb["total_pacientes"] = (
    resumen_pacientes_hosp_y_amb["pacientes_ambulatorios"]
    + resumen_pacientes_hosp_y_amb["pacientes_hospitalizados"]
    - resumen_pacientes_hosp_y_amb["pacientes_con_consulta_y_hosp"]
)